# Convert policies defined in Natural Language to Hashicorp Sentinel Language

## Introduction

In this notebook we will show you how to use Amazon Bedrock and Anthropic Claude for transforming policies defined in natural language to `Sentinel` language .


It will allow users to define policies in natural language without dependency on any domain-specific language (DSL). It integrates with terraform allowing organizations to define, implement, and enforce policies for infrastructure of their cloud and on-premises environments.


#### Use case

To demonstrate the natural language policy definition capabilities of Anthropic Claude via Amazon Bedrock, let's take the use case of creating IT security policies for a growing tech company.


#### Persona
You are Sarah, the newly appointed Chief Information Security Officer (CISO) at TechGrowth Inc., a rapidly expanding software company. As the company scales, you need to establish comprehensive IT security policies that align with industry standards and regulations. However, you're facing challenges:

1. Your team lacks expertise in policy writing in sentinel language.
2. You need policies that are clear and understandable to all employees, not just IT specialists.
3. The policies must be adaptable as the company grows and technology evolves.
4. You're under time pressure to implement these policies before the next board meeting.
5. You want to use Sentinel for policy enforcement. 

#### Implementation
To fulfill this use case, in this notebook we will show how to define policies in plain natural language and using Foundation Model (FM) to convert it into Hashicorp sentinel programming language.We will use the Anthropic Claude 3 Sonnet Foundation model using the Amazon Bedrock API and Langchain. 

#### Python 3.10

⚠  For this lab we need to run the notebook based on a Python 3.10 runtime. ⚠


## Installation

To run this notebook you would need to install dependencies - boto3, botocore and langchain.

Notice `capture` command below, this will suppress the output of pip installation commands. This will take approx about 3 - 5 mins to complete. You will not see any output as we are suppressing the output using `capture` command. If you would like to see the ouput, please comment out the `capture` command and run the cell. In this case, ignore `Warnings` and `Errors` you may see.

In [2]:
%%capture
%pip install --upgrade pip
%pip install boto3 --force-reinstall --quiet
%pip install botocore --force-reinstall --quiet
%pip install langchain --force-reinstall --quiet
%pip install langchain_aws --force-reinstall --quiet

## Kernel Restart

Restart the kernel with the updated packages that are installed through the dependencies above

In [3]:
# restart kernel
from IPython.core.display import HTML
HTML("<script>Jupyter.notebook.kernel.restart()</script>")

## Setup 

Import the necessary libraries

In [1]:
import json
import os
import sys
import boto3
import botocore
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
#from langchain_community.chat_models.bedrock import BedrockChat
from langchain_aws import ChatBedrock
from botocore.client import Config

In [2]:
from langchain.prompts import PromptTemplate

prompt = PromptTemplate(
    input_variables=["policy", "terraform_code"],
    template="""/
    <policy>{policy}<policy>/
    You are a Hashicorp Sentinel expert and you generate code in sentinel programming language from policies define in Natural language within <policy></policy> tags./
    Please generate the code within <code></code> tags.
    Please add your analysis and explanation with <explanation></explanation> tags.
    Think step by step and provide a detailed explanation for your answer./
    """
    )

## Define Policies as a Constitution in Natural Language
Here, we define all our policies as a constitution. Each policy is equivalent of a directive principle in this constitution.

In [3]:
policy = """
1. all EC2 instances should have tags/
"""

## Prompt template with policy and Terraform code as input

In [4]:
# format the prompt to add variable values
prompt_formatted_str: str = prompt.format(
    policy=policy)

## Initialization

Initiate Bedrock Runtime and BedrockChat

In [5]:
bedrock_config = Config(connect_timeout=120, read_timeout=120, retries={'max_attempts': 0})
bedrock_client = boto3.client('bedrock-runtime')

modelId = 'anthropic.claude-3-5-sonnet-20240620-v1:0' # change this to use a different version from the model provider

llm = ChatBedrock(model_id=modelId)

## Model Invocation and Response Generation

Invoke the model and visualize the response

In [6]:
response =llm.invoke(prompt_formatted_str)
print(response.content)

Certainly! I'll generate the Sentinel code based on the policy provided, add an explanation, and break down the thought process step by step.

<code>
import "tfplan/v2" as tfplan

// Main rule
main = rule {
    all ec2_instances_have_tags
}

// Rule to check if all EC2 instances have tags
ec2_instances_have_tags = rule {
    all tfplan.resources.aws_instance as _, instance {
        length(instance.change.after.tags) > 0
    }
}
</code>

<explanation>
Step-by-step thought process and explanation:

1. Policy Analysis:
   The policy states that "all EC2 instances should have tags". This means we need to check every EC2 instance resource in the Terraform plan and ensure it has at least one tag.

2. Import Statement:
   We start by importing the "tfplan/v2" module, which allows us to access the Terraform plan data. This is crucial for inspecting the resources that are being created or modified.

3. Main Rule:
   We define the main rule that will be evaluated when the policy is run. This ru

## Conclusion
You have now experimented with how to convert policies defined in natural language to Sentinel Policy

### Take aways
- Adapt this notebook to experiment with different Claude 3 models available through Amazon Bedrock. 
- Change the prompts to your specific usecase and evaluate the output of different models.
- Play with the token length to understand the latency and responsiveness of the service.
- Apply different prompt engineering principles to get better outputs.

## Thank You